## Attention-Model

This network spreading model considers the role of attention span in modeling the spread of memes on social networks. Agents maintain a limited attention span (queue) of their most recently observed memes. They draw from this queue to re-share memes.

In [22]:
from queue import Queue 
import networkx as nx
import collections
import random

In [23]:
class Agent:
    
    def __init__(self,ID,attention_span,interests):
        self.ID = ID
        self.attention_span = attention_span
        self.dq = collections.deque(maxlen=attention_span)
        self.interests = interests
        self.meme_share_chance = abs(random.random() - random.random()) #weighted towards 0
        self.meme_create_chance = abs(random.random() - random.random()) #weighted towards 0
        
    def add_meme(self,meme):
        self.dq.append(meme)
        
    def select_meme(self):
        if(len(self.dq) == 0):
            return None
        else:
            return random.choice(list(self.dq)) #Randomly select meme from queue
        #return item from queue at random, weighted by quality and similarity to agent interests
                
#Consider adding some variable(s) to capture agent interests/topics. 
#This could influence which meme in queue they select to share.

#When agents share memes, that meme gets added to the queue of their neighbors.
#The shared meme also remains in the queue of the agent who shared it. (thoughts on this?)

In [24]:
class Meme:
    
    def __init__(self,quality,topic):
        self.quality = quality
        self.topic = topic
    
#Memes could have some quality/novelty attribute and some topic attribute pulled from some distributions.
#These would be considered by the agent when selected which meme to share.

### Testing

In [25]:
testAgent = Agent(0,10,[])

In [26]:
testMeme = Meme(10,1)
testAgent.add_meme(testMeme)
print(testAgent.select_meme())

## Initialize Network

In [27]:
g = nx.DiGraph()

In [28]:
import gzip
import csv

following = {}

with gzip.open('twitter_combined.txt.gz', "rt", encoding="utf-8") as input_file:
    reader = csv.reader(input_file, delimiter=' ', quotechar='"')
    for record in reader:
        if(int(record[0]) not in following.keys()):
            following[int(record[0])] = [int(record[1])]
        else:
            following[int(record[0])].append(int(record[1]))

In [29]:
for node in following.keys():
    g.add_node(node)
for node in following.keys():
    for target in following[node]:
        g.add_edge(target,node) #Edge Direction from followee to follower - edges point outward toward followers

## Initialize Agents

In [30]:
agents = {}
for agent_id in following.keys():
    agent = Agent(agent_id, random.randint(5,10), [])
    agents[agent_id] = agent

## Initiate Meme Spread

In [32]:
def initiate_meme_spread(agents):
    for agent in agents:
        if random.random() < agent.meme_create_chance:
            new_meme = Meme(random.randint(0,10), random.randint(0,20))
            agent.add_meme(new_meme)
    return agents

## Iterate Meme Spread

In [33]:
def iterate_meme_spread(g, agents):
    for node in g.nodes():
        if random.random() < agent.meme_create_chance:
            new_meme = Meme(random.randint(0,10), random.randint(0,20))
            agents[node].add_meme(new_meme)
        if random.random() < agent.meme_share_chance:
            meme = agent.select_meme()
            if not meme == None:
                for neighbor in g.neighbors(node):
                    agents[neighbor].add_meme(meme)
    return agents

## Simulate Meme Spread

In [ ]:
def simulate_meme_spread(g, agents, iterations):
    agents = initiate_meme_spread(agents)
    for x in range(0, iterations):
        agents = iterate_meme_spread(g, agents)